# AI-Driven Early Prediction of Pulmonary Fibrosis Using Deep Learning


In [5]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 2e-3,
    "weight_decay": 3e-5,
    "batch_size": 64,
    "epochs": 300,
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "patience": 50,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv",
    "seed": 42
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# ==========================================
# STRATEGY C: RATIO-BASED DATA PREPROCESSING
# ==========================================
def preprocess_data_ratio(config):
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(
        columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'}
    )
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(
        columns={'Weeks': 'Base_Week'}
    )
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # Target: FVC Ratio
    train['FVC_Ratio'] = train['FVC'] / train['Base_FVC']
    
    # Interaction features
    train['FVC_Week_Interaction'] = train['Base_FVC'] * train['Relative_Weeks']
    train['Age_Week_Interaction'] = train['Age'] * train['Relative_Weeks']
    
    available_img_feats = [c for c in image_features if c in train.columns]
    
    if 'lung_vol_ml' in train.columns:
        train['LungVol_FVC_Ratio'] = train['lung_vol_ml'] / (train['Base_FVC'] + 1e-6)
    
    scaler = StandardScaler()
    
    num_cols = ['Age', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    interaction_cols = ['FVC_Week_Interaction', 'Age_Week_Interaction']
    
    if 'LungVol_FVC_Ratio' in train.columns:
        interaction_cols.append('LungVol_FVC_Ratio')
    
    all_num_cols = num_cols + interaction_cols
    train[all_num_cols] = scaler.fit_transform(train[all_num_cols])
    
    train['Base_FVC_Raw'] = train['Base_FVC']
    
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    feature_cols = all_num_cols + ['Sex', 'Smk_Ex', 'Smk_Cur']
    
    ratio_scaler = StandardScaler()
    ratio_scaler.fit(train[['FVC_Ratio']])
    train['FVC_Ratio_Scaled'] = ratio_scaler.transform(train[['FVC_Ratio']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols, ratio_scaler

# ==========================================
# MODEL
# ==========================================
class EnhancedQuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles, dropout=0.25):
        super().__init__()
        h1, h2, h3 = 256, 128, 64
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1), nn.BatchNorm1d(h1), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h1, h2), nn.BatchNorm1d(h2), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h2, h3), nn.BatchNorm1d(h3), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h3, len(quantiles))
        )
    
    def forward(self, x):
        return self.net(x)

def quantile_loss(preds, target, quantiles):
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# METRICS
# ==========================================
def calculate_metrics(y_true_fvc, q_preds_ratio, baseline_fvc):
    q20 = q_preds_ratio[:, 0] * baseline_fvc
    q50 = q_preds_ratio[:, 1] * baseline_fvc
    q80 = q_preds_ratio[:, 2] * baseline_fvc
    
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    delta = np.minimum(np.abs(y_true_fvc - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    mse = mean_squared_error(y_true_fvc, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_fvc, q50)
    r2 = r2_score(y_true_fvc, q50)
    rmae = mae / (np.mean(np.abs(y_true_fvc)) + 1e-6)
    
    return {
        'lll': np.mean(lll),
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'rmae': rmae,
        'r2': r2
    }

# ==========================================
# SIGMA CALIBRATION
# ==========================================
def calibrate_sigma_ratio(ratio_preds, scale_factor=0.85):
    preds_copy = ratio_preds.copy()
    q20, q50, q80 = preds_copy[:, 0], preds_copy[:, 1], preds_copy[:, 2]
    
    new_q20 = q50 - (q50 - q20) * scale_factor
    new_q80 = q50 + (q80 - q50) * scale_factor
    
    return np.column_stack([new_q20, q50, new_q80])

# ==========================================
# TRAINING WITH RATIO PREDICTION
# ==========================================
def train_ratio_model():
    df, features, ratio_scaler = preprocess_data_ratio(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    oof_indices = []
    oof_ratio_preds = []
    oof_trues_fvc = []
    oof_baselines = []
    
    print(f"\n🚀 Training RATIO PREDICTION Model")
    print(f"📊 Training on {len(df)} visits across {len(patients)} patients")
    print("="*80)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        print(f"\n{'='*80}")
        print(f"FOLD {fold+1}/{CONFIG['n_folds']}")
        print(f"{'='*80}")
        
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_ratio_scaled = torch.tensor(train_data['FVC_Ratio_Scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_val_fvc = val_data['FVC'].values
        val_baselines = val_data['Base_FVC_Raw'].values
        
        model = EnhancedQuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=20, verbose=False
        )
        
        best_lll = -float('inf')
        best_ratio_preds = None
        patience_counter = 0
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_ratio_scaled = model(X_train)
            
            loss = quantile_loss(preds_ratio_scaled, y_train_ratio_scaled, CONFIG['quantiles'])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            model.eval()
            with torch.no_grad():
                val_preds_ratio_scaled = model(X_val)
                val_preds_ratio = ratio_scaler.inverse_transform(val_preds_ratio_scaled.cpu().numpy())
                
                metrics = calculate_metrics(y_val_fvc, val_preds_ratio, val_baselines)
                lll = metrics['lll']
                
            scheduler.step(lll)
            
            if lll > best_lll:
                best_lll = lll
                best_ratio_preds = val_preds_ratio.copy()
                patience_counter = 0
                # 🎯 SAVE MODEL WITH REQUESTED NAME
                torch.save(model.state_dict(), f"optimised_model_fold{fold+1}.pth")
            else:
                patience_counter += 1
                
            if patience_counter >= CONFIG['patience']:
                break
        
        oof_indices.extend(val_data.index.tolist())
        oof_ratio_preds.append(best_ratio_preds)
        oof_trues_fvc.extend(y_val_fvc)
        oof_baselines.extend(val_baselines)
        
        fold_metrics = calculate_metrics(y_val_fvc, best_ratio_preds, val_baselines)
        print(f"\n📈 FOLD {fold+1} RESULTS:")
        print(f"  R²:   {fold_metrics['r2']:.4f}")
        print(f"  RMSE: {fold_metrics['rmse']:.2f} mL")
        print(f"  LLL:  {fold_metrics['lll']:.4f}")
    
    oof_ratio_preds = np.vstack(oof_ratio_preds)
    oof_trues_fvc = np.array(oof_trues_fvc)
    oof_baselines = np.array(oof_baselines)
    
    print("\n" + "="*80)
    print("🏁 BASELINE RATIO PREDICTION RESULTS")
    print("="*80)
    baseline_metrics = calculate_metrics(oof_trues_fvc, oof_ratio_preds, oof_baselines)
    print(f"R²:   {baseline_metrics['r2']:.4f}   {'✅' if baseline_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
    print(f"RMSE: {baseline_metrics['rmse']:.2f} mL {'✅' if baseline_metrics['rmse'] < 170 else '❌'} (Target < 170)")
    print(f"MAE:  {baseline_metrics['mae']:.2f} mL")
    print(f"LLL:  {baseline_metrics['lll']:.4f}   {'✅' if baseline_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
    
    print("\n" + "="*80)
    print("🧪 OPTIONAL: SIGMA CALIBRATION")
    print("="*80)
    
    best_lll = baseline_metrics['lll']
    best_factor = 1.0
    
    for scale in [0.95, 0.90, 0.85, 0.80, 0.75]:
        calibrated = calibrate_sigma_ratio(oof_ratio_preds, scale)
        metrics = calculate_metrics(oof_trues_fvc, calibrated, oof_baselines)
        
        improvement = "✅" if metrics['lll'] > best_lll else ""
        print(f"  σ×{scale:.2f} → LLL: {metrics['lll']:.4f} (RMSE: {metrics['rmse']:.2f}) {improvement}")
        
        if metrics['lll'] > best_lll:
            best_lll = metrics['lll']
            best_factor = scale
    
    if best_factor < 1.0:
        final_preds = calibrate_sigma_ratio(oof_ratio_preds, best_factor)
        final_metrics = calculate_metrics(oof_trues_fvc, final_preds, oof_baselines)
        
        print("\n" + "="*80)
        print("🏆 FINAL RESULTS (With Optimal Sigma)")
        print("="*80)
        print(f"Best σ scale factor: {best_factor:.2f}")
        print(f"\nR²:   {final_metrics['r2']:.4f}   {'✅' if final_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
        print(f"RMSE: {final_metrics['rmse']:.2f} mL {'✅' if final_metrics['rmse'] < 170 else '❌'} (Target < 170)")
        print(f"MAE:  {final_metrics['mae']:.2f} mL")
        print(f"LLL:  {final_metrics['lll']:.4f}   {'✅' if final_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
        
        print("\n" + "="*80)
        print("📊 IMPROVEMENT OVER BASELINE")
        print("="*80)
        print(f"RMSE: {baseline_metrics['rmse']:.2f} → {final_metrics['rmse']:.2f} ({final_metrics['rmse'] - baseline_metrics['rmse']:+.2f} mL)")
        print(f"LLL:  {baseline_metrics['lll']:.4f} → {final_metrics['lll']:.4f} ({final_metrics['lll'] - baseline_metrics['lll']:+.4f})")
    else:
        print("\n✅ No sigma calibration needed - baseline predictions are optimal!")
    
    print("="*80)

if __name__ == "__main__":
    train_ratio_model()

✅ Preprocessing Complete. Final Shape: (1549, 28)
✅ Features Used: 18

🚀 Training RATIO PREDICTION Model
📊 Training on 1549 visits across 176 patients

FOLD 1/5

📈 FOLD 1 RESULTS:
  R²:   0.9383
  RMSE: 195.41 mL
  LLL:  -6.4966

FOLD 2/5

📈 FOLD 2 RESULTS:
  R²:   0.9317
  RMSE: 232.93 mL
  LLL:  -6.8178

FOLD 3/5

📈 FOLD 3 RESULTS:
  R²:   0.8949
  RMSE: 243.82 mL
  LLL:  -6.8281

FOLD 4/5

📈 FOLD 4 RESULTS:
  R²:   0.9111
  RMSE: 228.20 mL
  LLL:  -6.6730

FOLD 5/5

📈 FOLD 5 RESULTS:
  R²:   0.9504
  RMSE: 201.94 mL
  LLL:  -6.6604

🏁 BASELINE RATIO PREDICTION RESULTS
R²:   0.9295   ✅ (Target > 0.88)
RMSE: 221.09 mL ❌ (Target < 170)
MAE:  152.18 mL
LLL:  -6.6942   ❌ (Target > -6.64)

🧪 OPTIONAL: SIGMA CALIBRATION
  σ×0.95 → LLL: -6.6982 (RMSE: 221.09) 
  σ×0.90 → LLL: -6.7053 (RMSE: 221.09) 
  σ×0.85 → LLL: -6.7158 (RMSE: 221.09) 
  σ×0.80 → LLL: -6.7306 (RMSE: 221.09) 
  σ×0.75 → LLL: -6.7506 (RMSE: 221.09) 

✅ No sigma calibration needed - baseline predictions are optimal!
